In [ ]:
pip install azure-storage-blob


Note: you may need to restart the kernel using dbutils.library.restartPython() to use updated packages.
  Using cached azure_storage_blob-12.18.1-py3-none-any.whl (392 kB)
  Using cached isodate-0.6.1-py2.py3-none-any.whl (41 kB)
  Using cached azure_core-1.29.4-py3-none-any.whl (192 kB)
  Using cached typing_extensions-4.8.0-py3-none-any.whl (31 kB)
  Attempting uninstall: typing-extensions
    Found existing installation: typing_extensions 4.3.0
    Not uninstalling typing-extensions at /databricks/python3/lib/python3.10/site-packages, outside environment /local_disk0/.ephemeral_nfs/envs/pythonEnv-6e402e55-769e-4755-be48-e3ea936bdfb4
    Can't uninstall 'typing_extensions'. No files were found to uninstall.
Note: you may need to restart the kernel using dbutils.library.restartPython() to use updated packages.


In [ ]:
from azure.storage.blob import BlobServiceClient

# Set up Azure Storage connection
azure_connection_string = "connection-string"
container_name = "github-issues-container"

# Create Azure Blob Service Client
blob_service_client = BlobServiceClient.from_connection_string(azure_connection_string)
container_client = blob_service_client.get_container_client(container_name)

# Create the container if it doesn't exist
container_client.create_container()

print(f"Container '{container_name}' created.")


Container 'github-issues-container' created.


In [ ]:
dbutils.widgets.text("year",'2023')
dbutils.widgets.text("month",'09')
dbutils.widgets.text("day",'01')
dbutils.widgets.text("hour",'01')
year = int(dbutils.widgets.get('year'))
month = int(dbutils.widgets.get('month'))
day = int(dbutils.widgets.get('day'))
hour = dbutils.widgets.get('hour')

In [ ]:
import requests
import gzip
from azure.storage.blob import BlobServiceClient, ContentSettings

# Set up Azure Storage connection
azure_connection_string = "DefaultEndpointsProtocol=https;AccountName=azureapexonintern;AccountKey=4uTPW2v4aVDvfu7Px75qKNpkM+sprr/DFqIPDuBj70anzdDzzgEfjubcqjAz/GOgvQG3gT6cSWWN+AStRwLI3Q==;EndpointSuffix=core.windows.net"
container_name = "github-issues-container"

# Set up GitHub Archive URL
github_archive_url = "https://data.gharchive.org/{year}-{month:02d}-{day:02d}-{hour}.json.gz"
# Replace these with the desired date and time
year = 2023
month = 8
day = 1
hour = 0

# Create Azure Blob Service Client
blob_service_client = BlobServiceClient.from_connection_string(azure_connection_string)
container_client = blob_service_client.get_container_client(container_name)

# Loop through desired time range
while True:
    url = github_archive_url.format(year=year, month=month, day=day, hour=hour)
    response = requests.get(url, stream=True)
    
    if response.status_code == 200:
        blob_name = f"{year}-{month:02d}-{day:02d}-{hour}.json.gz"
        blob_client = container_client.get_blob_client(blob_name)

        # Upload the gzipped content to Azure Storage
        blob_client.upload_blob(response.content, content_settings=ContentSettings(content_type="application/gzip"))
        
        print(f"Data from {year}-{month:02d}-{day:02d}-{hour} uploaded to Azure Storage.")

    hour += 1
    if hour == 24:
     hour = 0
    day += 1
    if day == 32:
     day = 1
    month += 1
    if month == 13:
     month = 1
     year += 1

    if year == 2023 and month == 9 and day == 1 and hour == 0:
        break
    
    # if year > 2023:  # Adjust the end year as needed
    #     break


Data from 2023-08-01-0 uploaded to Azure Storage.
Data from 2023-09-02-1 uploaded to Azure Storage.
